In [ ]:
%matplotlib inline


Created on Mon Mar  2 17:42:38 2020

@author: makinea1



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import trimesh
from mayavi import mlab
from bfieldtools.mesh_magnetics import magnetic_field_coupling
from bfieldtools.mesh_class import Conductor, StreamFunction


def field_disc(z, a):
    """ Bfield z-component of streamfunction psi=1/r on z-axis
        for a disk with radius "a" on the xy plane
    """

    coeff = 1e-7
    field = (a**2 + 2*z**2)/np.sqrt((a**2 + z**2)) - 2*abs(z)
    field *= coeff*2*np.pi

    return field


import pkg_resources
disc = trimesh.load(file_obj=pkg_resources.resource_filename('bfieldtools', 'example_meshes/unit_disc.stl'), process=False)
for i in range(4):
    disc = disc.subdivide()

disc.vertices -= disc.vertices.mean(axis=0)
c = Conductor(mesh_obj=disc)

s = -0.5*(disc.vertices[:,0]**2 + disc.vertices[:,1]**2)
ss = StreamFunction(s, c)

Np=50
z = np.linspace(-1, 10, Np)
points = np.zeros((Np, 3))
points[:,2] = z
mlab.triangular_mesh(*disc.vertices.T, disc.faces, scalars=s)
mlab.points3d(*points.T)

bfield_mesh = magnetic_field_coupling(disc, points, analytic=True) @ s

# Plot asymptotic error
Nr=100
drs = np.linspace(-0.0035, -0.0034, Nr)
from matplotlib import cm
colors = cm.viridis(np.linspace(0,1,Nr))
err = []
for c,dr in zip(colors,drs):
    err.append((abs(bfield_mesh[:,2] - field_disc(z, 1+dr))/field_disc(z, 1+dr))[-1])
plt.plot(drs, err)

# Solve zero-crossing
pp = np.polyfit(drs[:30], err[:30], deg=1)
dr0 = -pp[1]/pp[0]

#plt.plot(z, field_disc(z+0.001, 1+dr0))
#plt.plot(z, bfield_mesh[:,2])
plt.plot(z, (abs(bfield_mesh[:,2] - field_disc(z, 1+dr0))/field_disc(z, 1+dr0)))